Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [2]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 4.40 MiB | 6.00 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [3]:
cd gpt-2

/content/gpt-2


Install requirements

In [4]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 4.5MB/s 
     |████████████████████████████████| 604kB 19.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=f3c35ba3e4c0bd77d4dd8c2c8c57ca79d651da0b49ebd84d9813cf1e0eda2591
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533168 sha256=5be7e482ace5644c3df333c34cd10839c911037c6e7e484776a732b7e1d4e64c
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Download the model data

In [0]:
# !python3 download_model.py 117M

In [7]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 731kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 42.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 709kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:32, 43.3Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.54Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 41.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 39.0Mit/s]                                                       


encoding

In [0]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [9]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 

cp: cannot stat '/content/drive/My Drive/checkpoint/': No such file or directory


To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [0]:
# !wget https://www.gutenberg.org/files/98/98-0.txt


Start training, add --model_name '345M' to use 345 model

In [15]:
!PYTHONPATH=src './train.py' --dataset '/content/drive/My Drive/Colab Notebooks/Parser_Output.txt' --model_name '345M'


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-12-04 15:45:49.896939: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-12-04 15:45:49.897180: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2132f40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-04 15:45:49.897216: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-04 15:45:49.912631: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-04 15:45:49.987372: I tensorflow/stream_executor/cu

KeyboardInterrupt: ignored

Save our checkpoints to start training again later

In [0]:
!cp -r '/content/gpt-2/checkpoint/' '/content/drive/My Drive/'

Load your trained model for use in sampling below (117M or 345M)

In [0]:
# !cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [18]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name '345M'



2019-12-04 19:29:13.514599: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-04 19:29:13.563993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-12-04 19:29:13.564957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-12-04 19:29:13.592918: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2019-12-04 19:29:13.798411: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2019-12-04 19:29:13.939098: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10


To check flag descriptions, use:

In [0]:
!python3 src/interactive_conditional_samples.py -- --help

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [0]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

To check flag descriptions, use:

In [0]:
!python3 src/generate_unconditional_samples.py -- --help